In [1]:
from pdbfixer import PDBFixer
from simtk.openmm.app import PDBFile
from copy import deepcopy

In [2]:
fix_1brs=PDBFixer("1brs.pdb")

In [3]:
chains = list(fix_1brs.topology.chains())
num_chains = len(chains)

In [4]:
for chain in chains:
    print("Chain index {} with pdb id {}".format(chain.id,chain.index))

Chain index A with pdb id 0
Chain index B with pdb id 1
Chain index C with pdb id 2
Chain index D with pdb id 3
Chain index E with pdb id 4
Chain index F with pdb id 5
Chain index A with pdb id 6
Chain index B with pdb id 7
Chain index C with pdb id 8
Chain index D with pdb id 9
Chain index E with pdb id 10
Chain index F with pdb id 11


In [5]:
fix_1brs.removeChains([6,7,8,9,10,11])
chains = list(fix_1brs.topology.chains())
num_chains = len(chains)

In [6]:
fix_1brs.findMissingResidues()
fix_1brs.findNonstandardResidues()
fix_1brs.findMissingAtoms()

In [10]:
#print(fix_1brs.missingResidues)
#print(fix_1brs.nonstandardResidues)
#print(fix_1brs.missingAtoms)

In [11]:
fix_1brs.addMissingAtoms()

In [12]:
aux_dict={}
aux_dict["Barnase_A"]=0
aux_dict["Barnase_B"]=1
aux_dict["Barnase_C"]=2
aux_dict["Barstar_D"]=3
aux_dict["Barstar_E"]=4
aux_dict["Barstar_F"]=5

In [24]:
for clave in aux_dict.keys():
    chain_in=aux_dict[clave]
    chains_out=list(range(num_chains))
    chains_out.remove(chain_in)
    aux_1brs=deepcopy(fix_1brs)
    aux_1brs.removeChains(chains_out)
    pdbfilename = '1brs_'+clave+'_fixed.pdb'
    outfile = open(pdbfilename, 'w')
    PDBFile.writeFile(aux_1brs.topology, aux_1brs.positions, outfile)
    outfile.close()
    del(aux_1brs,chain_in,chains_out,outfile)

In [ ]:
for protein in aux_dict.keys():
    chain_in=aux_dict[protein]
    chains_out=list(range(num_chains))
    chains_out.remove(chain_in)
    aux_1brs=deepcopy(fix_1brs)
    aux_1brs.removeChains(chains_out)
    pdbfilename = '1brs_'+protein+'_fixed.pdb'
    outfile = open(pdbfilename, 'w')
    PDBFile.writeFile(aux_1brs.topology, aux_1brs.positions, outfile)
    outfile.close()
    del(aux_1brs,chain_in,chains_out,outfile)

In [ ]:
del(fix_1brs)

## Selecting chains